In [1]:
import pandas as pd
import numpy as np

In [2]:
best_annot = pd.read_excel('best_tau.xls', skiprows=1)

In [3]:
#lym ldl-c egfr fev/fvc

# Lympho LDL FEV1 FVC Smoke

best_annot[(best_annot['Trait'].isin(['Lympho', 'LDL', 'FEV1 FVC Smoke'])) & (best_annot['Pop'] == 'EUR')]

,Pop,Trait,BestAnnot,BestAnnot_Name,h2_inTop5%,h2se_inTop5,Unnamed: 6
11,EUR,Lympho,324,CD4 T cell[CD4-Positive T-Lymphocytes]:BCL6,0.54,0.05,NaN
27,EUR,LDL,615,Liver[Hep G2]:SRF,0.56,0.13,NaN
53,EUR,FEV1 FVC Smoke,697,Endothelial[HUVEC]:MAX,0.29,0.03,NaN


In [4]:
trait_annot_dict = {'INI30120': '324', 'INI20030780': '615', 'INI1003063': '697'}
trait_group_dict = {'INI30120': 'blood_count', 'INI20030780': 'blood_biochemistry', 'INI1003063': 'spirometry'}

In [91]:
annot_prism = pd.read_csv('/home/lucytian/data/1_Single_Cell_PRS/2_cV2F/ukb_genoHM3_mvp_eur_tissue_cV2F.annot.pvar.gz', sep='\t', compression='gzip')
annot_prism = annot_prism[~annot_prism['#CHROM'].isin(['X', 'Y', 'MT'])]
annot_prism['#CHROM'] = annot_prism['#CHROM'].astype(int)

for i in ['INI30120', 'INI20030780', 'INI1003063']:
    gwas = pd.read_csv(f'{gwas_d}/{trait_group_dict[i]}/pop_10PCs/WB_train_val/{i}.glm.tsv.gz', compression='gzip', delim_whitespace=True)
    gwas = gwas[~gwas['#CHROM'].isin(['X', 'Y', 'MT'])]
    gwas['#CHROM'] = gwas['#CHROM'].astype(int)
    gwas = annot_prism[['#CHROM', 'POS', 'ID', 'rsID']].merge(gwas,on=['#CHROM', 'POS', 'ID'])
    print(len(gwas))

    annot_name = f'Annot{trait_annot_dict[i]}'
    dfs = []
    for k in np.arange(1, 23):
        annot = pd.read_csv(f'/home/lucytian/lab4/IMPACT/IMPACT707_EUR_chr{str(k)}.annot.gz', compression='gzip', delim_whitespace=True)
        annot = annot[['CHR', 'BP', 'SNP', 'CM', annot_name]]
        annot = annot.rename(columns={'CHR': '#CHROM', 'BP': 'POS', 'SNP': 'rsID'})
        merged = gwas.merge(annot, on=['#CHROM', 'POS', 'rsID'])
        dfs.append(merged)

    top_snps = pd.concat(dfs)
    print(len(top_snps))
    threshold = top_snps[annot_name].quantile(0.95)
    top_snps[top_snps[annot_name] >= threshold]
    print(len(top_snps))
    top_snps.to_csv(f'top_SNPs/{i}.topsnps.tsv', index=False, header=True, sep='\t')

/tmp/ipykernel_20085/1893345654.py:1: DtypeWarning: Columns (0,9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  annot_prism = pd.read_csv('/home/lucytian/data/1_Single_Cell_PRS/2_cV2F/ukb_genoHM3_mvp_eur_tissue_cV2F.annot.pvar.gz', sep='\t', compression='gzip')
/tmp/ipykernel_20085/1893345654.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gwas = pd.read_csv(f'{gwas_d}/{trait_group_dict[i]}/pop_10PCs/WB_train_val/{i}.glm.tsv.gz', compression='gzip', delim_whitespace=True)


1300571
1188996
1188996


/tmp/ipykernel_20085/1893345654.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gwas = pd.read_csv(f'{gwas_d}/{trait_group_dict[i]}/pop_10PCs/WB_train_val/{i}.glm.tsv.gz', compression='gzip', delim_whitespace=True)


1300571
1188996
1188996


In [7]:
for i in ['INI30120', 'INI20030780', 'INI1003063']:
    df = pd.read_csv(f'top_SNPs/{i}.topsnps.tsv', sep='\t')
    df['P'] = 10 ** (-df['LOG10_P'])
    df.to_csv(f'top_SNPs/{i}.topsnps.tsv', index=False, header=True, sep='\t')

/tmp/ipykernel_236040/3357801725.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'top_SNPs/{i}.topsnps.tsv', sep='\t')
/tmp/ipykernel_236040/3357801725.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'top_SNPs/{i}.topsnps.tsv', sep='\t')


In [8]:
df

,#CHROM,POS,ID,rsID,REF,ALT,A1,TEST,OBS_CT,BETA,SE,T_STAT,LOG10_P,ERRCODE,ALT_FREQS,CM,Annot697,P
0,1,756604,1:756604:A:G,rs3131962,A,G,G,ADD,248147,0.00580944,0.004084,1.422360,0.809887,.,0.870326,0.490722,0.01680,0.154922
1,1,768448,1:768448:G:A,rs12562034,G,A,A,ADD,248163,-0.00432038,0.004465,-0.967688,0.477294,.,0.105303,0.495714,0.00000,0.333201
2,1,779322,1:779322:A:G,rs4040617,A,G,G,ADD,247965,-0.00412565,0.004115,-1.002610,0.500245,.,0.127473,0.500708,0.00177,0.316049
3,1,785989,1:785989:T:C,rs2980300,T,C,C,ADD,247523,0.00535317,0.004120,1.299440,0.712656,.,0.870990,0.507585,0.29100,0.193796
4,1,801536,1:801536:T:G,rs79373928,T,G,G,ADD,248452,-0.0201383,0.011316,-1.779630,1.124140,.,0.014875,0.587220,0.00321,0.075138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188991,22,51173542,22:51173542:T:C,rs4824147,T,C,C,ADD,248424,-0.003742,0.005701,-0.656296,0.291040,.,0.938687,75.081036,0.00725,0.511635
1188992,22,51174939,22:51174939:C:T,rs73174435,C,T,T,ADD,248282,0.003812,0.006081,0.626906,0.275133,.,0.053792,75.082492,0.00320,0.530722
1188993,22,51175626,22:51175626:A:G,rs3810648,A,G,G,ADD,247126,-0.009016,0.005744,-1.569670,0.933703,.,0.060835,75.083207,0.00493,0.116492
1188994,22,51185848,22:51185848:G:A,rs3865764,G,A,A,ADD,247962,-0.003222,0.006255,-0.515019,0.217140,.,0.949430,75.091114,0.00525,0.606541
